# **Outil de cartographie : carte de chaleur par département (France)**

Il est nécessaire de télécharger les limites géographiques des départements de France disponibles sur le Github : c'est le dossier "france_departements".

*Document d'entrée* : un fichier JSON contenant les informations sur les lieux obtenus par l'alignement avec Wikidata (voir notebook ```BaOIA_alignement_lieux_wikidata.ipynb```)

*Documents de sortie* : une carte de chaleur au format PNG et trois fichiers Excel utiles pour la création de la carte.

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''

## Installation des bibliothèques et connexion au compte Google Drive
!pip install pgeocode==0.3.0
!pip install geopandas==0.9.0
from google.colab import drive
import json
import pgeocode
import os
from openpyxl import load_workbook
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive :
chemin_vers_dossier_travail = '/content/drive/My Drive/lieux_wikidata/'#@param {type:"string"}

if not os.path.exists(chemin_vers_dossier_travail):
      os.makedirs(chemin_vers_dossier_travail)
os.chdir(chemin_vers_dossier_travail)

In [ ]:
#@markdown #Création de la carte de chaleur par département :

#@markdown ####Indiquer le chemin vers le fichier "France_departements-polygons.shp" contenu dans le dossier "france_departements":
chemin_fichier_shp = '/content/drive/My Drive/lieux_wikidata/france_departements/France_departements-polygon.shp' #@param {type:"string"}

#@markdown ####Indiquer le chemin vers le fichier JSON contenant les informations sur lieux :
chemin_fichier_json = "/content/drive/My Drive/lieux_wikidata/infos_lieux.json" #@param {type:"string"}

liste_codes = []

with open(chemin_fichier_json, "r") as f :
  contenu_json = json.load(f)
  for lieu in contenu_json:
    code = contenu_json[lieu]["Code Postal"]
    liste_codes.append(code)

nomi = pgeocode.Nominatim('fr')
df_tout = nomi.query_postal_code(liste_codes)
df_tout.to_excel('df_tout.xlsx', index = True, header=True)

## Compter nombre par département
df_nbr_depart=df_tout['county_name'].value_counts().rename_axis('nom').reset_index(name='county_name')
df_nbr_depart.to_excel('df_nbr_depart.xlsx', index = True, header=True)
df2_nbr_depart = pd.DataFrame(df_nbr_depart)

departs = gpd.read_file(chemin_fichier_shp)
departs.to_excel('voir_departs.xlsx', index = True, header=True)
departs.plot(figsize=(20,10))
merged = departs.set_index('nom').join(df2_nbr_depart.set_index('nom'))
merged = merged.reset_index()
merged = merged.fillna(0)
merged[['nom', 'geometry', 'county_name', 'code']]
fig, ax = plt.subplots(1, figsize=(20, 10))
ax.axis('off')
color = 'Reds'
vmin, vmax = 4, 93
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_title('Nombre de lieux cités par départements \n sur l\'ensemble du corpus de romans scolaires', fontdict={'fontsize': '13', 'fontweight' : '1'})
merged.plot('county_name', cmap=color, linewidth=0.8, ax=ax, edgecolor='0.8', figsize=(20,10))

#@markdown ####Indiquer le nom de la carte créée au format PNG :
nom_carte_png = 'carte_departements.png' #@param {type:"string"}

plt.savefig(nom_carte_png)